In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import re
import numpy as np

In [2]:
service=Service(ChromeDriverManager().install())

In [7]:
def get_jobs(keyword, num_jobs, verbose, slp_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder by changing service=(addres)
    #Chrome driver is being installed everytime the kernel is restart and saved as variable, 
    #not needing to provide location path
    driver = webdriver.Chrome(service=service, options=options)
    driver.set_window_size(1120, 1000)
    
    #Glassdoor website URL
    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    #If true, should be still looking for new jobs.
    while len(jobs) < num_jobs:  

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element(By.CLASS_NAME,"selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(1)

        try:
            driver.find_element(By.CSS_SELECTOR,'[alt="Close"]').click() #clicking to the X.
            print(' x out worked')
        except NoSuchElementException:
            print(' x out failed')
            pass

        
        #Gather all the job node elements from the left column of Glassdoor
        #These are the buttons we're going to click.
        job_buttons = driver.find_elements(By.XPATH, "//article[@id='MainCol']//ul/li[starts-with(@class, 'react-job-listing')]")  
        
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            #Click on each job node element
            webdriver.ActionChains(driver).move_to_element(job_button).click(job_button).perform()
            
            time.sleep(0.5)
            collected_successfully = False
            
            #Gather the basic information from the loaded job posting main page
            while not collected_successfully:
                try:
                    try:
                        company_name = driver.find_element(By.XPATH, './/div[@data-test="employerName"]').text
                    except NoSuchElementException:
                        company_name = None
                    try:
                        division = driver.find_element(By.XPATH, './/div[@class="division"]').text
                    except NoSuchElementException:
                        division = None
                    try:
                        location = driver.find_element(By.XPATH, './/div[@data-test="location"]').text
                    except NoSuchElementException:
                        location = None
                    try:
                        job_title = driver.find_element(By.XPATH, './/div[@data-test="jobTitle"]').text
                    except NoSuchElementException:
                        job_title = None
                    try:
                        job_description = driver.find_element(By.XPATH, './/div[@class="jobDescriptionContent desc"]').text
                    except NoSuchElementException:
                        job_description = None
                        
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                #Glassdoor salary estimate
                salary_estimate = driver.find_element(By.XPATH, "//div[starts-with(@class, 'salaryTab')]/div/div[2]/div").text
            except NoSuchElementException:
                salary_estimate = None #You need to set a "not found value. It's important."
            
            try:
                #Glassdoor employer review rating score
                rating = driver.find_element(By.XPATH, './/div[@id="employerStats"]/div[1]/div').text
            except NoSuchElementException:
                rating = None #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Division: {}".format(division))
                print("Location: {}".format(location))


            #Gather additional information about the company from Glassdoor company overview section
            try:
                driver.find_element(By.XPATH, './/div[@id="CompanyContainer"]')

                try:
                    size = driver.find_element(By.XPATH, './/*[@id="EmpBasicInfo"]/div[1]/div/div[1]/span[2]').text
                except NoSuchElementException:
                    size = None

                try:
                    founded = driver.find_element(By.XPATH, './/*[@id="EmpBasicInfo"]/div[1]/div/div[2]/span[2]').text
                except NoSuchElementException:
                    founded = None

                try:
                    type_of_ownership = driver.find_element(By.XPATH, './/*[@id="EmpBasicInfo"]/div[1]/div/div[3]/span[2]').text
                except NoSuchElementException:
                    type_of_ownership = None

                try:
                    industry = driver.find_element(By.XPATH, './/*[@id="EmpBasicInfo"]/div[1]/div/div[4]/span[2]').text
                except NoSuchElementException:
                    industry = None

                try:
                    sector = driver.find_element(By.XPATH, './/*[@id="EmpBasicInfo"]/div[1]/div/div[5]/span[2]').text
                except NoSuchElementException:
                    sector = None

                try:
                    revenue = driver.find_element(By.XPATH, './/*[@id="EmpBasicInfo"]/div[1]/div/div[6]/span[2]').text
                except NoSuchElementException:
                    revenue = None


            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = None
                size = None
                founded = None
                type_of_ownership = None
                industry = None
                sector = None
                revenue = None

                
            if verbose:
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Division" : division,             
            "Location" : location,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue})
            #add job to jobs
                
        #Clicking on the "next page" button
        try:
            driver.find_element(By.XPATH, "//article[@id='MainCol']//button[starts-with(@class, 'nextButton')]").click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break
   
    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [13]:
# Can change the salary part to get the entire range estimate rather than average
# Company descriptors do not necessarily include the same factors listed (line 113-145)
# As the section uses div indexing, the values may not match with correct column
# Can instead perform an evaluation based on the sub-heading

# https://stackoverflow.com/questions/3655549/xpath-containstext-some-string-doesnt-work-when-used-with-node-with-more
# //*[contains(text(),'ABC')]

# try:
#    driver.findElement.textcontains('industry')
#    try:
#        industry = driver.findElement(industry).text
#    except NoSuchElementException:
#        industry = None
# except NoSuchElementException:
#     industry = None

In [9]:
#df = get_jobs('data scientist',1000, False, 15)
df = get_jobs('data scientist',10, False, 15)

df.to_csv('glassdoor_jobs.csv', index = False)

KeyboardInterrupt: 

In [12]:
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Division,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Data Scientist,None,Position Summary...\nWhat you'll do...\nData S...,3.3,Walmart\n3.3,None,"Sunnyvale, CA",10000+ Employees,1962,Company - Public,General Merchandise & Superstores,Retail & Wholesale,$10+ billion (USD)
2,Data Scientist,"$125,491 /yr (est.)",Data science at the National Security Agency (...,3.5,National Security Agency\n3.5,None,"Fort Meade, MD",Unknown,Government,National Agencies,Government & Public Administration,Unknown / Non-Applicable,None
3,Data Scientist,None,Come join a growing financial technology compa...,4.4,Continental Finance Company\n4.4,None,"Wilmington, DE",51 to 200 Employees,2005,Company - Private,Banking & Lending,Financial Services,$5 to $25 million (USD)
4,"Data Scientist, Marketing & Online (Remote)","$125,000 /yr (est.)",Position Purpose:\nThe Data Scientist is respo...,3.8,The Home Depot\n3.8,None,"Atlanta, GA",10000+ Employees,1978,Company - Public,Home Furniture & Housewares Stores,Retail & Wholesale,$10+ billion (USD)
5,Data Analyst,None,Job Description: Data Analyst\n\nWhy Guarantee...,2.9,Guaranteed Rate\n2.9,None,Remote,5001 to 10000 Employees,2000,Company - Private,Banking & Lending,Financial Services,$500 million to $1 billion (USD)
6,Machine Learning Engineer/Data Scientist,$82.50 /hr (est.),Machine Learning Engineer/Data Scientist\nDesc...,4.0,"Daman, Inc.\n4.0",None,Remote,51 to 200 Employees,1995,Company - Private,Information Technology Support Services,Information Technology,$25 to $100 million (USD)
7,Senior Data Scientist - Remote,"$150,000 /yr (est.)",Required experience:\n· PhD or Master Degree +...,None,FSS Government Solutions,None,Remote,1 to 50 Employees,Company - Private,Research & Development,Management & Consulting,Unknown / Non-Applicable,None
8,Data Scientist,"$110,000 /yr (est.)","SupplyHouse.com, an e-commerce heating, plumbi...",3.9,SupplyHouse\n3.9,None,Remote,501 to 1000 Employees,2004,Company - Private,Other Retail Stores,Retail & Wholesale,Unknown / Non-Applicable
9,None,None,None,None,None,None,None,None,None,None,None,None,None


In [11]:
df.dtypes

Job Title            object
Salary Estimate      object
Job Description      object
Rating               object
Company Name         object
Division             object
Location             object
Size                 object
Founded              object
Type of ownership    object
Industry             object
Sector               object
Revenue              object
dtype: object